In [35]:
import warnings
warnings.filterwarnings('ignore')

import polars as pl
import pandas as pd
import numpy as np

import backtrader as bt

from poly_utils import get_markets
from backtrader_plotting import Bokeh
from backtrader.feeds import PandasData

from backtrader_plotting.schemes import Blackly
from bokeh.plotting import output_file, save

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
df = pl.scan_csv("processed/trades.csv").collect(streaming=True)

df = df.with_columns(
    pl.col("timestamp").str.to_datetime().alias("timestamp")
)

In [57]:
markets = get_markets()

Loaded 119554 markets from markets.csv
Combined total: 119277 unique markets (sorted by createdAt)


In [58]:
markets = markets.with_columns(
    pl.col("createdAt").str.to_datetime().alias("createdAt")
)

In [63]:
markets = markets.with_columns(
    pl.col('createdAt').dt.replace_time_zone(None)
)

In [73]:
target_id = markets.filter(markets['question'].str.contains('Will Donald Trump win the 2024 US Presidential Election')).sort('volume').row(0, named=True)['id']

In [74]:
sel_df = df.filter(pl.col('market_id') == target_id)

In [75]:
sel_df

timestamp,market_id,maker,taker,nonusdc_side,maker_direction,taker_direction,price,usd_amount,token_amount,transactionHash
datetime[μs],i64,str,str,str,str,str,f64,f64,f64,str
2024-01-05 02:36:24,253591,"""0xd42f6a1634a3707e27cbae14ca96…","""0xf0b049379bbd6399ad1c6704345a…","""token2""","""BUY""","""SELL""",0.58,22.095238,38.095237,"""0x7fea7cb4b1d8e2af697578ee7d6b…"
2024-01-05 02:36:24,253591,"""0xf0b049379bbd6399ad1c6704345a…","""0xc5d563a36ae78145c45a50134d48…","""token1""","""BUY""","""SELL""",0.42,99.999999,238.095237,"""0x7fea7cb4b1d8e2af697578ee7d6b…"
2024-01-05 02:36:24,253591,"""0x8698edbefd013db6d087e3d09eef…","""0xf0b049379bbd6399ad1c6704345a…","""token1""","""SELL""","""BUY""",0.42,84.0,200.0,"""0x7fea7cb4b1d8e2af697578ee7d6b…"
2024-01-05 03:33:10,253591,"""0x8698edbefd013db6d087e3d09eef…","""0x9d84ce0306f8551e02efef168047…","""token2""","""SELL""","""BUY""",0.59,118.0,200.0,"""0x186fb98a63dbbb74dd4e5d167fe7…"
2024-01-05 03:33:10,253591,"""0xd42f6a1634a3707e27cbae14ca96…","""0x9d84ce0306f8551e02efef168047…","""token2""","""SELL""","""BUY""",0.59,5.9,10.0,"""0x186fb98a63dbbb74dd4e5d167fe7…"
…,…,…,…,…,…,…,…,…,…,…
2024-11-06 15:20:33,253591,"""0xb79dd020f60b7d09bc801c5a6055…","""0xc5d563a36ae78145c45a50134d48…","""token1""","""BUY""","""SELL""",0.999,0.34965,0.35,"""0x2b6365ceac2aa48a4f1975b21b6a…"
2024-11-06 15:20:35,253591,"""0x2fb0f88ef5ba40e799b996e6f07b…","""0xd53d94afcb32eb1db663118e5f83…","""token1""","""BUY""","""SELL""",0.998,94.18126,94.37,"""0xcef4b27fdb54c6fd8330cb5cb591…"
2024-11-06 15:20:35,253591,"""0xd53d94afcb32eb1db663118e5f83…","""0xc5d563a36ae78145c45a50134d48…","""token1""","""SELL""","""BUY""",0.998,94.18126,94.37,"""0xcef4b27fdb54c6fd8330cb5cb591…"


In [76]:
sel_df = sel_df[['timestamp', 'price', 'usd_amount', 'nonusdc_side']]

In [77]:
sel_df = sel_df.to_pandas()

In [78]:
sel_df['price'] = np.where(sel_df['nonusdc_side'] == 'token2', 1 - sel_df['price'], sel_df['price']) #standardize it for ease

In [79]:
sel_df = sel_df[['timestamp', 'price', 'usd_amount']]

In [80]:
sel_df = sel_df.set_index('timestamp')

# Resample to 1-minute bars
ohlcv = sel_df.resample('10min').agg({
    'price': ['first', 'max', 'min', 'last'],
    'usd_amount': 'sum'
})

# Flatten column names
ohlcv.columns = ['open', 'high', 'low', 'close', 'volume']

# Reset index if you want timestamp as a column
ohlcv = ohlcv.reset_index()


In [81]:
ohlcv = ohlcv.dropna()


In [82]:
ohlcv = ohlcv[ohlcv['timestamp'] >= "2024-10-01"]

In [83]:
ohlcv

,timestamp,open,high,low,close,volume
38865,2024-10-01 00:00:00,0.489,0.489,0.488,0.489,1.242784e+04
38866,2024-10-01 00:10:00,0.488,0.489,0.488,0.488,2.018100e+02
38867,2024-10-01 00:20:00,0.489,0.489,0.488,0.489,2.011670e+03
38868,2024-10-01 00:30:00,0.489,0.489,0.488,0.488,6.926000e+02
38869,2024-10-01 00:40:00,0.488,0.489,0.488,0.488,4.223999e+01
...,...,...,...,...,...,...
44137,2024-11-06 14:40:00,0.998,0.999,0.997,0.997,8.303148e+05
44138,2024-11-06 14:50:00,0.998,0.999,0.997,0.997,4.766331e+05
44139,2024-11-06 15:00:00,0.997,0.999,0.997,0.998,3.205917e+05
44140,2024-11-06 15:10:00,0.998,0.999,0.997,0.997,1.297173e+06


In [84]:
ohlcv['close'].plot() #sanity check

<IPython.core.display.Javascript object>

<Axes: >

In [115]:
ohlcv = ohlcv[ohlcv['timestamp'] != '2024-10-24 21:00:00'] #remove broken data for aesthetic

In [116]:
class PandasDataFeed(PandasData):
    params = (
        ('datetime', 'timestamp'),
        ('open', 'open'),
        ('high', 'high'),
        ('low', 'low'),
        ('close', 'close'),
        ('volume', 'volume'),
        ('openinterest', -1),  # Set to -1 to disable
    )

In [120]:
class SimpleMAStrategy(bt.Strategy):
    params = (
        ('fast_period', 50),
        ('slow_period', 200),
        ('stop_loss', 0.02),  # 2% stop loss
        ('trailing_stop', False),  # Enable trailing stop
        ('risk_percent', 0.95),  # Use 95% of portfolio
    )
    
    def __init__(self):
        self.fast_ma = bt.indicators.SimpleMovingAverage(
            self.data.close, period=self.params.fast_period
        )
        self.slow_ma = bt.indicators.SimpleMovingAverage(
            self.data.close, period=self.params.slow_period
        )
        self.crossover = bt.indicators.CrossOver(self.fast_ma, self.slow_ma)
        self.crossover.plotinfo.plot = False
        self.crossover.plotinfo.plotmaster = self.data
        
        self.order = None
        self.buy_price = None
    
    def next(self):
        # Check stop loss first if we have a position
        if self.position:
            # Calculate stop loss price
            stop_price = self.buy_price * (1 - self.params.stop_loss)
            
            # Exit on stop loss
            if self.data.close[0] <= stop_price:
                self.close()
                return
            
            # Exit on MA crossover
            if self.crossover < 0:
                self.close()
                return
        
        # Entry logic
        if not self.position:
            if self.crossover > 0:
                # Calculate position size
                cash = self.broker.getcash() * self.params.risk_percent
                size = cash / self.data.close[0]
                self.buy(size=size)
                self.buy_price = self.data.close[0]
    
    def notify_order(self, order):
        if order.status in [order.Completed]:
            if order.isbuy():
                self.buy_price = order.executed.price

In [128]:
# Create cerebro engine
cerebro = bt.Cerebro()

# Prepare data - ensure timestamp is datetime and set as index
ohlcv_bt = ohlcv.copy()
ohlcv_bt['timestamp'] = pd.to_datetime(ohlcv_bt['timestamp'])  # Convert to datetime
ohlcv_bt = ohlcv_bt.set_index('timestamp')  # Set as index

# Add data feed using built-in PandasData
data = bt.feeds.PandasData(
    dataname=ohlcv_bt,
    openinterest=-1
)
cerebro.adddata(data, name='Will Donald Trump win the 2024 US Presidential Election')

# Add strategy
cerebro.addstrategy(SimpleMAStrategy)

# Set initial cash
cerebro.broker.setcash(10000.0)

# Set commission (0.1%)
cerebro.broker.setcommission(commission=0)

# Print starting conditions
print(f'Starting Portfolio Value: {cerebro.broker.getvalue():.2f}')

# Run backtest
cerebro.run()

# Print final results
print(f'Final Portfolio Value: {cerebro.broker.getvalue():.2f}')

Starting Portfolio Value: 10000.00
Final Portfolio Value: 20949.23


In [ ]:
scheme = Blackly()



b = Bokeh(
    style='bar', 
    plot_mode='single', 
    scheme=scheme,
    output_mode='save',
    filename='processed/plot.html'
)
cerebro.plot(b)
